# 12.4b: Fast Sigma Sweep (Compute-and-Discard)

**Goal:** Generate synthetic snowballs, compute statistics, and immediately discard embeddings.

## The Fast Approach

Instead of saving 21.5 GB of embeddings to disk, we:
1. Generate batch of embeddings on GPU
2. Compute all statistics we care about
3. **Throw away embeddings** and keep only the tiny statistics
4. Repeat

This is ~100× faster because we skip all I/O overhead.

## Output

- CSV with comprehensive statistics per trial
- Element-wise distribution plot (histogram + Q-Q plot)
- Scatter plot: σ vs singleton count (if sweeping)

## Parameters

In [ ]:
# Experiment parameters
SIGMA = 3.281911e-03   # Measured from Qwen's dead tokens
N_TRIALS = 1000        # Number of independent snowballs to generate
N_TOKENS = 2100        # Qwen's dead token count
HIDDEN_DIM = 2560      # Qwen's embedding dimension
BATCH_SIZE = 10        # Trials per batch (adjust based on GPU memory)

# Epsilon (bfloat16 precision at Qwen's scale)
EPSILON = 5.9604645e-05  # From 01.3c
TOUCHING_THRESHOLD = 2 * EPSILON  # Adjacency criterion

# Output
OUTPUT_CSV = "../data/analysis/synthetic_comprehensive_sigma3.281911e-03_n10000.csv"
OUTPUT_DIR = "../data/results/"

# Figure settings
DPI = 150
COLORMAP = 'inferno'

RANDOM_SEED = 42

## Imports

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from safetensors.torch import load_file
from pathlib import Path
from tqdm.auto import tqdm
from scipy import stats as scipy_stats
import networkx as nx
import gc
import time

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Setup device
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("✓ MPS (Metal Performance Shaders) available")
else:
    device = torch.device('cpu')
    print("⚠ MPS not available, using CPU only")

print(f"  Device: {device}")

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("✓ Imports complete")

✓ MPS (Metal Performance Shaders) available
  Device: mps
✓ Imports complete


## Load Qwen Centroid

In [3]:
print("Loading Qwen black hole centroid...\n")

centroid_data = load_file("../data/tensors/black_hole_centroid_qwen3_4b.safetensors")
qwen_centroid = centroid_data['centroid'].to(torch.float32).to(device)

print(f"✓ Centroid loaded to {device}")
print(f"  Shape: {qwen_centroid.shape}")
print(f"  Norm: {qwen_centroid.norm().item():.6f}")

Loading Qwen black hole centroid...

✓ Centroid loaded to mps
  Shape: torch.Size([2560])
  Norm: 0.166061


## Statistics Functions

In [4]:
def compute_gini_coefficient(populations):
    """Compute Gini coefficient of inequality."""
    if len(populations) <= 1:
        return 0.0
    
    sorted_pops = torch.sort(populations.float())[0]
    n = len(sorted_pops)
    
    indices = torch.arange(1, n + 1, dtype=torch.float32)
    numerator = 2 * torch.sum(indices * sorted_pops)
    denominator = n * torch.sum(sorted_pops)
    
    if denominator == 0:
        return 0.0
    
    gini = (numerator / denominator) - (n + 1) / n
    return gini.item()


def compute_l_inf_stats(unique_vectors, epsilon):
    """Compute L∞ (Chebyshev) distance statistics."""
    n = len(unique_vectors)
    
    if n <= 1:
        return {'max_l_inf': 0.0, 'mean_l_inf': 0.0, 'median_l_inf': 0.0}
    
    # Compute pairwise L∞ distances using cdist (memory efficient)
    l_inf_matrix = torch.cdist(unique_vectors, unique_vectors, p=float('inf'))
    
    # Exclude diagonal (self-distances)
    mask = ~torch.eye(n, dtype=torch.bool)
    l_inf_values = l_inf_matrix[mask]
    
    # Normalize by epsilon
    l_inf_values = l_inf_values / epsilon
    
    return {
        'max_l_inf': l_inf_values.max().item(),
        'mean_l_inf': l_inf_values.mean().item(),
        'median_l_inf': l_inf_values.median().item(),
    }


def compute_topology(unique_vectors, threshold):
    """Compute graph topology statistics using NetworkX."""
    n = len(unique_vectors)
    
    if n == 0:
        return {
            'n_components': 0,
            'n_isolated': 0,
            'largest_component_size': 0,
            'largest_component_density': 0.0,
            'global_density': 0.0,
        }
    
    if n == 1:
        return {
            'n_components': 1,
            'n_isolated': 1,
            'largest_component_size': 1,
            'largest_component_density': 1.0,
            'global_density': 1.0,
        }
    
    # Compute pairwise L∞ distances using cdist (memory efficient)
    l_inf_matrix = torch.cdist(unique_vectors, unique_vectors, p=float('inf'))
    
    # Build adjacency matrix (exclude self-loops)
    adjacency = (l_inf_matrix <= threshold) & (~torch.eye(n, dtype=torch.bool))
    
    # Convert to NetworkX graph
    G = nx.Graph()
    G.add_nodes_from(range(n))
    edges = torch.nonzero(adjacency, as_tuple=False).tolist()
    G.add_edges_from(edges)
    
    # Connected components
    components = list(nx.connected_components(G))
    component_sizes = sorted([len(c) for c in components], reverse=True)
    
    n_components = len(components)
    largest_size = component_sizes[0] if component_sizes else 0
    
    # Isolated nodes (degree = 0)
    n_isolated = sum(1 for node in G.nodes() if G.degree(node) == 0)
    
    # Density of largest component
    if largest_size > 1:
        largest_component = max(components, key=len)
        subgraph = G.subgraph(largest_component)
        n_edges = subgraph.number_of_edges()
        max_edges = largest_size * (largest_size - 1) // 2
        largest_density = n_edges / max_edges if max_edges > 0 else 0.0
    else:
        largest_density = 1.0 if largest_size == 1 else 0.0
    
    # Global density
    n_edges = G.number_of_edges()
    max_edges = n * (n - 1) // 2
    global_density = n_edges / max_edges if max_edges > 0 else 0.0
    
    return {
        'n_components': n_components,
        'n_isolated': n_isolated,
        'largest_component_size': largest_size,
        'largest_component_density': largest_density,
        'global_density': global_density,
    }


def compute_trial_statistics(embeddings, epsilon, touching_threshold):
    """Compute all statistics for a single trial."""
    # Move to CPU for torch.unique (not MPS-compatible)
    embeddings_cpu = embeddings.cpu()
    
    # Find unique vectors and counts
    unique_vectors, inverse_indices, counts = torch.unique(
        embeddings_cpu,
        dim=0,
        return_inverse=True,
        return_counts=True
    )
    
    # Basic counts
    n_tokens = len(embeddings)
    n_unique = len(unique_vectors)
    
    # Black holes (count >= 2)
    black_hole_mask = counts >= 2
    n_black_holes = black_hole_mask.sum().item()
    black_hole_population = counts[black_hole_mask].sum().item() if n_black_holes > 0 else 0
    
    # Singletons (count == 1)
    singleton_mask = counts == 1
    n_singletons = singleton_mask.sum().item()
    
    # Population stats
    total_population = counts.sum().item()
    
    stats = {
        'n_tokens': n_tokens,
        'n_unique': n_unique,
        'n_black_holes': n_black_holes,
        'n_singletons': n_singletons,
        'total_population': total_population,
        'black_hole_population': black_hole_population,
    }
    
    # Per-black-hole statistics
    if n_black_holes > 0:
        bh_populations = counts[black_hole_mask]
        sorted_pops = torch.sort(bh_populations, descending=True)[0]
        
        stats['largest_bh'] = sorted_pops[0].item()
        stats['smallest_bh'] = sorted_pops[-1].item()
        stats['mean_bh_size'] = bh_populations.float().mean().item()
        stats['median_bh_size'] = bh_populations.float().median().item()
        stats['top2_population'] = sorted_pops[:2].sum().item() if len(sorted_pops) >= 2 else sorted_pops[0].item()
        stats['gini_coefficient'] = compute_gini_coefficient(bh_populations)
    else:
        stats['largest_bh'] = 0
        stats['smallest_bh'] = 0
        stats['mean_bh_size'] = 0.0
        stats['median_bh_size'] = 0.0
        stats['top2_population'] = 0
        stats['gini_coefficient'] = 0.0
    
    # Spatial extent
    l_inf_stats = compute_l_inf_stats(unique_vectors, epsilon)
    stats.update(l_inf_stats)
    
    # Topology
    topology_stats = compute_topology(unique_vectors, touching_threshold)
    stats.update(topology_stats)
    
    return stats


print("✓ Statistics functions defined")

✓ Statistics functions defined


## Generate and Compute Statistics (Streaming)

In [5]:
print(f"\nGenerating {N_TRIALS:,} synthetic snowballs (compute-and-discard)...")
print(f"  σ = {SIGMA:.3e}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Shape per trial: [{N_TOKENS}, {HIDDEN_DIM}]\n")

results = []
n_batches = (N_TRIALS + BATCH_SIZE - 1) // BATCH_SIZE

# Track all element values for distribution plot
all_elements_sample = []  # Sample for plotting (don't keep all 5.4M values)
sample_every_n_trials = max(1, N_TRIALS // 1000)  # Keep ~1000 trials worth

start_time = time.time()

for batch_idx in tqdm(range(n_batches), desc="Processing batches"):
    # Determine batch size
    start_idx = batch_idx * BATCH_SIZE
    end_idx = min(start_idx + BATCH_SIZE, N_TRIALS)
    current_batch_size = end_idx - start_idx
    
    # Generate batch on device
    noise = torch.randn(current_batch_size, N_TOKENS, HIDDEN_DIM, dtype=torch.float32, device=device) * SIGMA
    embeddings_batch = qwen_centroid.unsqueeze(0).unsqueeze(0) + noise
    
    # Quantize to bfloat16 then back to float32
    embeddings_batch = embeddings_batch.to(torch.bfloat16).to(torch.float32)
    
    # Process each trial in batch
    for trial_idx in range(current_batch_size):
        embeddings = embeddings_batch[trial_idx]
        
        # Sample elements for distribution plot
        global_trial_idx = start_idx + trial_idx
        if global_trial_idx % sample_every_n_trials == 0:
            # Keep centered noise (not absolute embeddings)
            centered = embeddings - qwen_centroid.unsqueeze(0)
            all_elements_sample.append(centered.flatten().cpu())
        
        # Compute all statistics
        stats = compute_trial_statistics(embeddings, EPSILON, TOUCHING_THRESHOLD)
        stats['trial_id'] = global_trial_idx
        
        results.append(stats)
    
    # Free batch memory
    del noise, embeddings_batch
    gc.collect()

elapsed_time = time.time() - start_time

print(f"\n✓ Processed {N_TRIALS:,} trials in {elapsed_time:.1f}s")
print(f"  ({N_TRIALS/elapsed_time:.1f} trials/sec)")


Generating 1,000 synthetic snowballs (compute-and-discard)...
  σ = 3.282e-03
  Batch size: 10
  Shape per trial: [2100, 2560]



Processing batches:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Save Results to CSV

In [ ]:
df_results = pd.DataFrame(results)

output_path = Path(OUTPUT_CSV)
df_results.to_csv(output_path, index=False)

print(f"\n✓ Results saved to {output_path}")
print(f"  Rows: {len(df_results):,}")
print(f"  Columns: {len(df_results.columns)}")
print(f"  Size: {output_path.stat().st_size / 1024:.1f} KB")

## Quick Statistics Summary

In [ ]:
print(f"\n{'='*80}")
print(f"SUMMARY STATISTICS (n = {N_TRIALS:,} trials)")
print(f"{'='*80}\n")

# Key metrics
key_metrics = [
    'n_unique', 'n_black_holes', 'n_singletons',
    'black_hole_population', 'largest_bh',
    'gini_coefficient', 'max_l_inf',
    'n_components', 'largest_component_density', 'global_density'
]

for metric in key_metrics:
    mean = df_results[metric].mean()
    std = df_results[metric].std()
    min_val = df_results[metric].min()
    max_val = df_results[metric].max()
    
    if std < 0.001:
        print(f"{metric:30s} mean={mean:.1f},  std={std:.3f},  range=[{min_val:.1f}, {max_val:.1f}]")
    else:
        print(f"{metric:30s} {mean:.1f} ± {std:.1f}")

print(f"\n{'='*80}")

## Figure 1: Element-wise Distribution

In [ ]:
# Concatenate sampled elements
all_elements = torch.cat(all_elements_sample).numpy()

print(f"\nElement-wise distribution (sampled from {len(all_elements_sample)} trials)")
print(f"  Total elements: {len(all_elements):,}")
print(f"  Mean: {all_elements.mean():.3e}")
print(f"  Std: {all_elements.std():.3e}")

# Compute distribution stats
measured_mean = all_elements.mean()
measured_std = all_elements.std()
skewness = scipy_stats.skew(all_elements)
kurtosis = scipy_stats.kurtosis(all_elements)

print(f"  Skewness: {skewness:+.3f}")
print(f"  Kurtosis: {kurtosis:+.3f}")

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(14, 5), dpi=DPI)

# Histogram
ax = axes[0]
ax.hist(all_elements, bins=100, color='steelblue', alpha=0.7, edgecolor='black', density=True)

# Overlay Gaussian with measured parameters
x = np.linspace(all_elements.min(), all_elements.max(), 1000)
gaussian = scipy_stats.norm.pdf(x, loc=measured_mean, scale=measured_std)
ax.plot(x, gaussian, 'r-', linewidth=2, label=f'N({measured_mean:.2e}, {measured_std:.2e}²)')

ax.set_xlabel('Value', fontsize=11, fontweight='bold')
ax.set_ylabel('Density', fontsize=11, fontweight='bold')
ax.set_title('Element-wise Distribution', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Q-Q plot
ax = axes[1]
scipy_stats.probplot(all_elements, dist="norm", plot=ax)
ax.set_title('Q-Q Plot (Normal Distribution)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.suptitle(f'Synthetic Snowball Distribution (σ = {SIGMA:.3e})\n(n = {len(all_elements_sample):,} sampled trials, d = 2,560 dimensions)',
             fontsize=13, fontweight='bold', y=1.02)

plt.tight_layout()

# Save
output_path = Path(OUTPUT_DIR) / f"synthetic_distribution_sigma{SIGMA:.3e}.png"
plt.savefig(output_path, dpi=DPI, bbox_inches='tight')
print(f"\n✓ Saved: {output_path}")

plt.show()

## Summary

In [ ]:
file_size_kb = output_path.stat().st_size / 1024

print(f"\n{'='*80}")
print(f"FAST GENERATION COMPLETE")
print(f"{'='*80}")
print(f"Trials: {N_TRIALS:,}")
print(f"σ = {SIGMA:.3e}")
print(f"\nGeneration time: {elapsed_time:.1f}s ({N_TRIALS/elapsed_time:.1f} trials/sec)")
print(f"\nOutput CSV: {OUTPUT_CSV}")
print(f"Size: {file_size_kb:.1f} KB (vs ~21 GB if we saved embeddings!)")
print(f"\nSpeedup: ~{(21 * 1024 * 1024) / file_size_kb:.0f}× smaller")
print(f"{'='*80}")